# Init stuffs

### Input data

In [15]:
filename = "./input_1.txt"
test_filename = "./input_2.txt"

with open(file=filename) as f:
    data = f.read().splitlines()

with open(file=test_filename) as f:
    test_data = f.read().splitlines()

# Pt. 1

In [166]:
# Directions: 0 == north, 1 == east, 2 == south, 3 == west
DIRECTION_DELTAS = {0: (0, -1), 1: (1, 0), 2: (0, 1), 3: (-1, 0)}
DIRECTION_CHANGES = {
    "/": {0: 1, 1: 0, 2: 3, 3: 2},
    "\\": {0: 3, 1: 2, 2: 1, 3: 0},
    ".": {0: 0, 1: 1, 2: 2, 3: 3},
}
SPLITS = {
    "-": {0: [1, 3], 1: [1], 2: [1, 3], 3: [3]},
    "|": {0: [0], 1: [0, 2], 2: [2], 3: [0, 2]}
}

def simple_propagation(active_beams: list[tuple], energized_tiles: list[tuple], next_beam_loc: list[int], next_direction: int) -> tuple[list[tuple], list[tuple]]:
    next_beam = tuple(next_beam_loc+[next_direction])
    if next_beam not in energized_tiles:
        energized_tiles.append(next_beam)
        active_beams.append(next_beam)
    return active_beams, energized_tiles

def propagate_beam(
    grid: list[str], active_beams: list[tuple], energized_tiles: list[tuple]
) -> tuple[list[tuple], list[tuple]]:
    current_x, current_y, current_dir = active_beams.pop(0)
    next_beam_loc = [
        current_x + DIRECTION_DELTAS[current_dir][0],
        current_y + DIRECTION_DELTAS[current_dir][1],
    ]
    if (
        min(next_beam_loc) < 0 or max(next_beam_loc) > len(grid) - 1
    ):  # propagates outside of grid
        return active_beams, energized_tiles

    next_grid_value = grid[next_beam_loc[1]][next_beam_loc[0]]
    if next_grid_value in DIRECTION_CHANGES.keys(): # easy propagations
        next_direction = DIRECTION_CHANGES[grid[next_beam_loc[1]][next_beam_loc[0]]][current_dir]
        return simple_propagation(active_beams=active_beams, energized_tiles=energized_tiles, next_beam_loc=next_beam_loc, next_direction=next_direction)
    else: # splitter behavior
        splitter_behavior = SPLITS[next_grid_value][current_dir]
        if len(splitter_behavior) == 1:
            return simple_propagation(active_beams=active_beams, energized_tiles=energized_tiles, next_beam_loc=next_beam_loc, next_direction=splitter_behavior[0])
        else:
            energized_tiles.append(tuple(next_beam_loc+[current_dir]))
            for next_action in splitter_behavior:
                active_beams.append(tuple(next_beam_loc+[next_action]))
            return active_beams, energized_tiles


In [181]:
active_beams = [(-1, 0, 1)]  # tuple of x,y,direction entries of active beams
energized_tiles = []

while active_beams:
    active_beams, energized_tiles = propagate_beam(grid=data, active_beams=active_beams, energized_tiles=energized_tiles)
len(set([(x, y) for x, y, _ in energized_tiles]))

6994

# Pt. 2 (brute force to the win)

In [185]:
tiles = []
for i in range(len(data)):
    active_beams = [(-1, i, 1)]
    energized_tiles = []
    while active_beams:
        active_beams, energized_tiles = propagate_beam(grid=data, active_beams=active_beams, energized_tiles=energized_tiles)
    tiles.append(len(set([(x, y) for x, y, _ in energized_tiles])))

    active_beams = [(i, -1, 2)]
    energized_tiles = []
    while active_beams:
        active_beams, energized_tiles = propagate_beam(grid=data, active_beams=active_beams, energized_tiles=energized_tiles)
    tiles.append(len(set([(x, y) for x, y, _ in energized_tiles])))

    active_beams = [(len(data), i, 3)]
    energized_tiles = []
    while active_beams:
        active_beams, energized_tiles = propagate_beam(grid=data, active_beams=active_beams, energized_tiles=energized_tiles)
    tiles.append(len(set([(x, y) for x, y, _ in energized_tiles])))

    active_beams = [(i, len(data), 0)]
    energized_tiles = []
    while active_beams:
        active_beams, energized_tiles = propagate_beam(grid=data, active_beams=active_beams, energized_tiles=energized_tiles)
    tiles.append(len(set([(x, y) for x, y, _ in energized_tiles])))

print(max(tiles), len(tiles))

7488 440


# Runtime of that (&uarr;) cell is about 2m30s